In [1]:
import pandas as pd 
import numpy as np
from doepy import build
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import random
import pickle
import copy

In [2]:
from IISEPaper import categorise
from IISEPaper import get_labeled_index
from IISEPaper import get_feature_label
from IISEPaper import get_labeled_set
from IISEPaper import get_training_Set
from IISEPaper import get_unlabeled_set
from IISEPaper import c_prediction
from sklearn.gaussian_process import GaussianProcessClassifier
from IISEPaper import predicted_region_divide
from IISEPaper import get_labeled_feasible_sample
from IISEPaper import get_feasible_rep
from IISEPaper import get_div_term
from IISEPaper import find_indices
from IISEPaper import get_select_index
from IISEPaper import GP_predict
from IISEPaper import labeled_fea_sample
from IISEPaper import remove_label
from IISEPaper import get_index_div

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
model = GaussianProcessClassifier()

In [5]:
data = pd.read_csv('dataset1101.csv')
ds,X_feature,y_ground,y_r = get_feature_label(data)
full_index = list(range(y_ground.shape[0]))
index_file = np.load('data1_10_index.npy',allow_pickle = True)

In [6]:
batch_size = 3

In [7]:
acc = []
result=[]
for n in range(len(index_file)):
    labeled_index= list(index_file[n])
    full_index = list(range(y_ground.shape[0]))
    x_labeled = X_feature[labeled_index]
    y_labeled = y_ground[labeled_index]
    y_r_labeled = y_r[labeled_index]
    unlabeled_index = remove_label(full_index,labeled_index)
    unlabeled_feature = X_feature[unlabeled_index]
    r_selected = y_r[labeled_index]
    labeled_feasible_sample, labeled_infeasible_sample,labeled_r = labeled_fea_sample(x_labeled,y_labeled,r_selected)
    r = [min(labeled_r)-0]

    l=0
    f1=[]
    y_c_pred , y_prob , y_c_uncertainty = c_prediction(x_labeled, y_labeled, unlabeled_feature,model)
    y_c = y_ground.copy()
    y_c[unlabeled_index] = y_c_pred
    f1.append(accuracy_score(y_ground, y_c))
    
    # Here choose min (r)>0 and len (r) < 50 if run the optimization objective 
    #while min(r) >0 and len(r) <50:
    
    # Change the maximum number of iteration based on your setting
    
    while l<20:
        selection_index = random.sample(unlabeled_index,batch_size)
        labeled_index= labeled_index+selection_index 
        x_labeled = X_feature[labeled_index]
        unlabeled_index = remove_label(full_index,labeled_index)
        unlabeled_feature = X_feature[unlabeled_index]
        y_labeled = y_ground[labeled_index]
        r_selected = y_r[labeled_index]
        y_r_labeled = y_r[labeled_index]
        labeled_feasible_sample, labeled_infeasible_sample,labeled_r = labeled_fea_sample(x_labeled,y_labeled,r_selected)
        y_c_pred , y_prob , y_c_uncertainty = c_prediction(x_labeled, y_labeled, unlabeled_feature,model)
        y_c = y_ground.copy()
        y_c[unlabeled_index] = y_c_pred
        f1.append(accuracy_score(y_ground, y_c))
        r.append(min(labeled_r)-0)
        l=l+1
    result.append(r)
    acc.append(f1)
    print('Round {} finished'.format(n))

Round 0 finished
Round 1 finished
Round 2 finished
Round 3 finished
Round 4 finished
Round 5 finished
Round 6 finished
Round 7 finished
Round 8 finished
Round 9 finished
Round 10 finished
Round 11 finished
Round 12 finished
Round 13 finished
Round 14 finished
Round 15 finished
Round 16 finished
Round 17 finished
Round 18 finished
Round 19 finished
Round 20 finished


In [8]:
from itertools import zip_longest

temp = np.array(list(zip_longest(*result, fillvalue=0))).T

In [9]:
#optimization performance
opt_mean = temp.mean(axis=0)
opt_std = temp.std(axis=0)

In [10]:
#active learning performance
mean_al=np.array(acc).mean(axis=0)
std_al = np.array(acc).std(axis=0)

In [11]:
mean_al

array([0.74584289, 0.74881731, 0.74810912, 0.75261324, 0.75672077,
       0.75842044, 0.75839211, 0.75921362, 0.75921362, 0.76210306,
       0.76555905, 0.76960993, 0.77317923, 0.77587037, 0.78074276,
       0.78094105, 0.78230078, 0.78402878, 0.78722982, 0.78921277,
       0.78949605])

In [12]:
std_al

array([0.03487051, 0.03416555, 0.03604146, 0.03441147, 0.03341678,
       0.03234378, 0.02996057, 0.03062733, 0.03100792, 0.03309726,
       0.03193428, 0.03166732, 0.0322808 , 0.02989729, 0.02848445,
       0.02686116, 0.02602347, 0.02544392, 0.02550865, 0.02569296,
       0.02545505])